In [1]:
from datetime import datetime as dt
import pandas as pd
import time
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

/tmp/ipykernel_290392/3204776876.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def extract_data(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')    
    return df
    
df = extract_data(url_by_city)
df.head()


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2021-03-01,2021-03-31,30,place,6,13096,f,"North Charleston, SC",North Charleston,South Carolina,SC,Multi-Family (2-4 Unit),4,348500.0,1.323333,0.372047,285000.0,0.140000,NaN,86.978131,0.015688,-0.100776,155.388471,0.019913,NaN,4.0,0.333333,1.000000,3.0,0.0,2.0,4.0,0.333333,NaN,6.0,0.0,-0.40,1.5,-0.5,-3.5,46.0,-1.0,-29.0,1.057512,0.146625,0.092585,0.5,0.166667,0.500000,NaN,NaN,NaN,0.333333,0.0,0.333333,"Charleston, SC",16700,2024-04-21 14:27:01
1,2020-03-01,2020-03-31,30,place,6,18164,f,"Strasburg, MO",Strasburg,Missouri,MO,All Residential,-1,19500.0,NaN,NaN,NaN,NaN,NaN,17.922794,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,NaN,NaN,NaN,NaN,"Kansas City, MO",28140,2024-04-21 14:27:01
2,2021-03-01,2021-03-31,30,place,6,14293,f,"Odell, IL",Odell,Illinois,IL,Single Family Residential,6,110000.0,-0.140625,0.222222,57450.0,-0.662059,-0.361667,64.942213,0.191284,0.429716,41.279273,-0.617074,-0.332652,2.0,1.000000,-0.333333,1.0,0.0,0.0,2.0,1.000000,1.00,2.0,1.0,0.00,1.0,0.0,0.3,31.0,22.0,-186.0,1.000000,-0.071130,0.035964,0.0,-1.000000,-0.333333,0.500000,-0.5,NaN,1.000000,0.0,0.000000,"Pontiac, IL",38700,2024-04-21 14:27:01
3,2023-04-01,2023-04-30,30,place,6,499,f,"Arcadia, IN",Arcadia,Indiana,IN,Single Family Residential,6,154750.0,NaN,0.312831,212450.0,NaN,0.089767,133.867604,NaN,1.019687,131.314545,NaN,0.245968,2.0,NaN,0.000000,3.0,NaN,0.0,4.0,NaN,1.00,1.0,NaN,0.00,0.5,NaN,0.0,62.0,NaN,48.0,0.947368,NaN,-0.066104,0.0,NaN,-0.500000,NaN,NaN,NaN,1.000000,NaN,0.333333,"Indianapolis, IN",26900,2024-04-21 14:27:01
4,2015-02-01,2015-02-28,30,place,6,5788,f,"Fairview, NC",Fairview,North Carolina,NC,Single Family Residential,6,425000.0,NaN,0.107492,200000.0,NaN,0.459854,100.710900,NaN,-0.153595,102.197241,NaN,0.284611,1.0,NaN,-0.500000,NaN,NaN,NaN,1.0,NaN,-0.75,9.0,NaN,-0.55,9.0,NaN,-1.0,508.0,NaN,433.0,0.955056,NaN,-0.011525,0.0,NaN,0.000000,0.111111,NaN,0.061111,NaN,NaN,NaN,"Charlotte, NC",16740,2024-04-21 14:27:01


In [4]:
print('Num of rows:', len(df))
print('Num of cols:', len(df.columns))

Num of rows: 5294100
Num of cols: 58


In [5]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def lambda_handler(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')
   
   
     # Remove commas from the 'city' column
    df['city'] = df['city'].str.replace(',', '')
    cols = ['period_begin','period_end','period_duration', 'region_type', 'region_type_id', 'table_id',
    'is_seasonally_adjusted', 'city', 'state', 'state_code', 'property_type', 'property_type_id',
    'median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold',
    'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list', 'sold_above_list', 'parent_metro_region_metro_code', 
    'last_updated']
    df = df[cols]
    
    df = df.dropna()
    
    #let's change the period_begin and period_end to date time object and extract years and month.
    df['period_begin'] = pd.to_datetime(df['period_begin'])
    df['period_end'] = pd.to_datetime(df['period_end'])
    
    df["period_begin_in_years"] = df['period_begin'].dt.year
    df["period_end_in_years"] = df['period_end'].dt.year

    df["period_begin_in_months"] = df['period_begin'].dt.month
    df["period_end_in_months"] = df['period_end'].dt.month
    
    #let's map the month number to their respective month name.
    month_dict = {
        "period_begin_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec",
        },
        "period_end_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec"
        }}
    df = df.replace(month_dict)
    print('Num of rows:', len(df))
    print('Num of cols:', len(df.columns))
    return df

In [6]:
df = lambda_handler(url_by_city)
df.head()

Num of rows: 4175462
Num of cols: 28


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,city,state,state_code,property_type,property_type_id,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,parent_metro_region_metro_code,last_updated,period_begin_in_years,period_end_in_years,period_begin_in_months,period_end_in_months
0,2021-03-01,2021-03-31,30,place,6,13096,f,North Charleston,South Carolina,SC,Multi-Family (2-4 Unit),4,348500.0,285000.0,86.978131,155.388471,4.0,6.0,1.5,46.0,1.057512,0.5,16700,2024-04-21 14:27:01,2021,2021,Mar,Mar
2,2021-03-01,2021-03-31,30,place,6,14293,f,Odell,Illinois,IL,Single Family Residential,6,110000.0,57450.0,64.942213,41.279273,2.0,2.0,1.0,31.0,1.000000,0.0,38700,2024-04-21 14:27:01,2021,2021,Mar,Mar
3,2023-04-01,2023-04-30,30,place,6,499,f,Arcadia,Indiana,IN,Single Family Residential,6,154750.0,212450.0,133.867604,131.314545,2.0,1.0,0.5,62.0,0.947368,0.0,26900,2024-04-21 14:27:01,2023,2023,Apr,Apr
4,2015-02-01,2015-02-28,30,place,6,5788,f,Fairview,North Carolina,NC,Single Family Residential,6,425000.0,200000.0,100.710900,102.197241,1.0,9.0,9.0,508.0,0.955056,0.0,16740,2024-04-21 14:27:01,2015,2015,Feb,Feb
5,2022-10-01,2022-10-31,30,place,6,23777,f,Las Flores,California,CA,Single Family Residential,6,1080000.0,1389950.0,508.225617,527.389128,5.0,6.0,1.2,68.0,0.970314,0.2,11244,2024-04-21 14:27:01,2022,2022,Oct,Oct


In [7]:
# Saving DataFrame to CSV
df.to_csv(df, index=False)

print(f"CSV file '{df}' has been saved successfully.")

TypeError: argument of type 'method' is not iterable

In [13]:
#number of years in our dataset
# sorted(df["period_begin_in_years"].unique())

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [ ]:
# download file
# df.to_csv('real_estate_w209_byCity.csv', index=False)